In [ ]:
#!pip install simple_exact_diagonalization_routines
from simple_exact_diagonalization_routines.local_matrix_class import *
from scipy.linalg import expm
import matplotlib.pyplot as plt

# System size
L = 9

# Create objects of Pauli operators
X = X_class(L)
Y = Y_class(L)
Z = Z_class(L)
S_plus = S_plus_class(L)
S_minus = S_minus_class(L)

# Trivial initialization of the right size matrix
H_TFIM = 0*np.eye( 2**L )

for x in range(L-1):
    H_TFIM = H_TFIM + 2*X.at(x+1).dot(X.at(x+2)) + Z.at(x+1)
H_TFIM = H_TFIM + Z.at(L)

# Trivial initialization of the right size matrix
hamiltonian_name = "TLFIM_L_" +str(L)
H_TLFIM = 0*np.eye( 2**L )

for x in range(L-1):
    H_TLFIM = H_TLFIM + 2*X.at(x+1).dot(X.at(x+2)) + Z.at(x+1)+ X.at(x+1)
H_TLFIM = H_TLFIM + Z.at(L)    + X.at(L)

In [ ]:
class helpers:
    @staticmethod
    def commutator( A, B ):
            return A.dot( B ) - B.dot( A )

    @staticmethod
    def delta( A ):
        return np.diag( A.diagonal() )

    @staticmethod
    def sigma( A ):
        return A - helpers.delta( A )
    @staticmethod
    def GWW_dbr( s, H ):
      from scipy.linalg import expm
      V = expm( s* helpers.commutator( helpers.delta( H), H))
      return V.dot( H.dot( V.T.conj() ) )

    def run_param_rc( self, fontsize = 30):
        import matplotlib.pyplot as plt
        plt.rcParams['axes.labelsize'] = fontsize
        plt.rcParams['axes.titlesize'] = fontsize
        plt.rcParams['font.size'] = fontsize
        #set_matplotlib_formats('pdf', 'png')
        plt.rcParams['savefig.dpi'] = 75
        plt.rcParams['lines.linewidth'] = 2.0
        plt.rcParams['lines.markersize'] = 8
        plt.rcParams['legend.fontsize'] = 12
        plt.rcParams['legend.labelspacing'] = .3
        plt.rcParams['legend.columnspacing']= .3
        plt.rcParams['legend.handletextpad']= .1
        plt.rcParams['text.usetex'] = True
        plt.rcParams['font.family'] = "serif"
        plt.rcParams['font.serif'] = "cm"
flow = helpers()
flow.run_param_rc(fontsize=15)


In [ ]:
norms_sigma_GC = []
norms_sigma_GWW = []
norms_sigma_GC2 = []
norms_sigma_GC3 = []
flow_times = np.linspace(0.001,.12,35)
for i,s in enumerate(flow_times):

    sp = np.sqrt(s)
    V_d = expm( 1j*sp*flow.delta( H_TFIM) )
    V_H = expm( 1j*sp* H_TFIM)
    V_GC = V_d.dot(V_H).dot(V_d.conj().T).dot(V_H.conj().T)
    H_s = V_GC.conj().T.dot(H_TFIM).dot(V_GC)

    sp = np.sqrt(s/2)
    V_d = expm( 1j*(sp*flow.delta( H_TFIM)).real )
    V_H = expm( 1j*(sp* H_TFIM).real)
    V_GC = V_d.dot(V_H).dot(V_d.conj().T).dot(V_H.conj().T)
    V_GC2 = V_GC.dot(V_GC)

    H_s2 = V_GC2.conj().T.dot(H_TFIM).dot(V_GC2)
    sp = np.sqrt(s/4)
    V_d = expm( 1j*sp*flow.delta( H_TFIM) )
    V_H = expm( 1j*sp* H_TFIM)

    V_GC = V_d.dot(V_H).dot(V_d.conj().T).dot(V_H.conj().T)
    V_GC4 = V_GC.dot(V_GC).dot(V_GC).dot(V_GC)


    H_s3 = V_GC4.conj().T.dot(H_TFIM).dot(V_GC4)
    norms_sigma_GC.append(np.linalg.norm(flow.sigma(H_s)))
    norms_sigma_GC2.append(np.linalg.norm(flow.sigma(H_s2)))
    norms_sigma_GC3.append(np.linalg.norm(flow.sigma(H_s3)))

    norms_sigma_GWW.append(np.linalg.norm(flow.sigma(flow.GWW_dbr(s, H=H_TFIM))))

In [ ]:

f=plt.figure(figsize = ( 6,4 ))
plt.plot(flow_times,norms_sigma_GWW, label=r'$ e^{-s\hat W}$')
plt.plot(flow_times,norms_sigma_GC, label=r'$\hat V^{\mathrm{(GC)}}_{\sqrt{s}}$')
plt.plot(flow_times,norms_sigma_GC2, label=r'$\biggl(\hat V^{\mathrm{(GC)}}_{\sqrt{s/2}}\biggr)^2$')
plt.plot(flow_times, norms_sigma_GC3, label=r'$\biggl(\hat V^{\mathrm{(GC)}}_{\sqrt{s/4}}\biggr)^4$')
plt.legend(loc="best")
plt.xlabel(r'DBI rotation duration $s$')
plt.title(r'Norm off-diagonal restriction $\vert\vert\sigma(\hat H_0(s))\vert\vert_\mathrm{HS}$')

norm_mins = [norms_sigma_GC[0],min(norms_sigma_GWW),min(norms_sigma_GC),min(norms_sigma_GC2),min(norms_sigma_GC3)]
y_labels_rounded = [ round(y, 1 ) for y in norm_mins ]
plt.grid()
plt.yticks(y_labels_rounded)
save_path = 'Fig_more_steps_L_'+str(L)+'.pdf'
plt.rcParams["figure.figsize"] = [80, 3.50]
plt.rcParams["figure.autolayout"] = True
plt.savefig( save_path, format='pdf')
plt.show()
